In [1]:
# change to the root directory of the project
%cd ..
%pip install -e .

/home/ubuntu/code/decentralized-search


Obtaining file:///home/ubuntu/code/decentralized-search
  Preparing metadata (setup.py) ... done
  Attempting uninstall: openkaito
    Found existing installation: openkaito 0.2.4
    Uninstalling openkaito-0.2.4:
      Successfully uninstalled openkaito-0.2.4
  Running setup.py develop for openkaito
Note: you may need to restart the kernel to use updated packages.


In [2]:
import asyncio
import os
import random
from datetime import datetime, timedelta

import bittensor as bt
import openai
from dotenv import load_dotenv

from openkaito.crawlers.twitter.apidojo import ApiDojoTwitterCrawler
from openkaito.evaluation.evaluator import Evaluator
from openkaito.protocol import StructuredSearchSynapse
from openkaito.tasks import (generate_author_index_task,
                             generate_structured_search_task)
from openkaito.utils.version import get_version

load_dotenv()
bt.logging.set_debug(True)
bt.logging.set_trace(True)

## Query Miners & Evaluation

In [3]:
wallet = bt.wallet(name="<wallet name>", hotkey="<hotkey name>")
metagraph = bt.metagraph(5, sync=True)

miners_uid = [128, 23, 242, 8, 232, 98, 130, 245, 252, 128]

with open("twitter_usernames.txt") as f:
    twitter_usernames = f.read().strip().splitlines()

# search_query = generate_author_index_task(10, 2)
# search_query.timeout = 90

# search_query = generate_structured_search_task()
search_query = StructuredSearchSynapse(
    query_string="TAO",
    size=5,
    later_than_timestamp=(datetime.now() - timedelta(days=365)).timestamp(),
    author_usernames=random.sample(twitter_usernames, 1000),
    version=get_version(),
)
# set a short timeout to direct query the miners, forcing them not to crawl data on request
# search_query.timeout = 10
# or set a long timeout to allow the miners to crawl data on request
search_query.timeout = 90
bt.logging.info(search_query.json())

async def send():
    dendrite = bt.dendrite(wallet=wallet)
    responses = await dendrite(
        axons=[metagraph.axons[uid] for uid in miners_uid],
        synapse=search_query,
        deserialize=True,
        timeout=search_query.timeout,
    )
    print(responses)
    return responses

responses = asyncio.run(send())
bt.logging.success(responses)


#### Evaluation

# need to setup OPENAI_API_KEY and APIFY_API_KEY in .env
llm_client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    organization=os.getenv("OPENAI_ORGANIZATION"),
    max_retries=3,
)

# for integrity check
twitter_crawler = ApiDojoTwitterCrawler(os.environ["APIFY_API_KEY"])

evaluator = Evaluator(llm_client, twitter_crawler)

evaluator.evaluate(search_query, responses)

2024-04-03 06:08:54.352 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-04-03 06:08:54.353 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-04-03 06:08:54.353 |     WARNING      | In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it.
2024-04-03 06:08:54.729 |       INFO       | Connected to finney network and wss://entrypoint-finney.opentensor.ai:443.
2024-04-03 06:09:18.343 |       INFO       | {"name": "StructuredSearchSynapse", "timeout": 90.0, "total_size": 0, "header_size": 0, "dendrite": {"status_code": null, "status_message": null, "process_time": null, "ip": null, "port": null, "version": null, "nonce": null, "uuid": null, "hotkey": null, "signature": null}, "axon": {"status_code": null, "status_messag

[None, [{'id': '1774003437476233243', 'text': "🚨🇱🇧BREAKING: UN DELEGATES' CAR HIT IN LEBANON BY IDF\n\nAn Israeli strike in southern Lebanon reportedly hit a car carrying UN observers.\n\n4 members of the United Nations Truce Supervision Organization were injured as a result of being bombed near Rmeish, south of  Lebanon.\n\nSource: Reuters\nPhoto: Archived", 'created_at': '2024-03-30T09:19:13+00:00', 'username': 'MarioNawfal', 'url': 'https://x.com/MarioNawfal/status/1774003437476233243', 'quote_count': 0, 'reply_count': 29, 'retweet_count': 10, 'favorite_count': 27}, {'id': '1773837839311986961', 'text': "🇺🇸ABDUCTION OF CHILD PREVENTED BY THE LOCAL\xa0COMMUNITY\n\nA community in California rallied together to prevent the abduction of an eight-year-old boy by fugitive Christopher Aaron Higginbotham during a family's deli opening celebration. \n\nThe boy's father, along with bystanders, quickly intervened, stopping Higginbotham who falsely claimed the child as his own. \n\nHigginbotham

tensor([0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000,
        0.0000])